<a href="https://colab.research.google.com/github/AraKchrUser/Automation-of-citizens-appeals/blob/master/pps_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 

# PPS1

In [ ]:
!pip install transformers
!pip install deeppavlov
!python -m deeppavlov install squad_bert

In [ ]:
!pip install pymorphy2
!pip install pymorphy2-dicts
!pip install DAWG-Python

In [37]:
import numpy as np

import deeppavlov
from deeppavlov import configs
from deeppavlov.core.common.file import read_json
from deeppavlov.core.commands.infer import build_model
from deeppavlov import configs, train_model


### SQUAD

In [ ]:
# Squad
model_ru = build_model(configs.squad.squad_ru_rubert, download=True)

/usr/local/lib/python3.7/dist-packages/cryptography/hazmat/backends/openssl/x509.py:17: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,
2022-02-03 05:39:47.922 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/deeppavlov_data/squad_model_ru_rubert.tar.gz to /root/.deeppavlov/squad_model_ru_rubert.tar.gz
100%|██████████| 659M/659M [00:32<00:00, 20.3MB/s]
2022-02-03 05:40:21.423 INFO in 'deeppavlov.core.data.utils'['utils'] at line 272: Extracting /root/.deeppavlov/squad_model_ru_rubert.tar.gz archive into /root/.deeppavlov/models
2022-02-03 05:40:29.662 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz to /root/.deeppavlov/downloads/rubert_cased_L-12_H-768_A-12_v1.tar.gz
100%|██████████| 666M/666M [00:32<00:00, 20.3MB/s]












The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.





Instructions for updating:
Use standard file APIs to check for files with this prefix.


2022-02-03 05:41:40.640 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/squad_ru_bert/model_rubert]



INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/squad_ru_bert/model_rubert


In [ ]:
text = ''' В этом году в Благовещенске участвуют в программе более 100 дворов. 12.24.2021 завершаются ремонтные 
работы на улице Ленина 165. Академический театр драмы обновляется в соответствии с решением губернатора. Также 
13.34.2035 начинаются работы на Богдана-Хмельницкого 156, построят 153 ловочек, 8 тренажеров.'''

print(model_ru([text], ['Когда заканчиваются работы на улице Ленина 165'])[0])
print(model_ru([text], ['Сколько дворов участвуют в программе в Благовещенск']))
print(model_ru([text], ['Что будет сделано там на Богдана-Хмельницкого 156']))

['12.24.2021']
[['более 100 дворов'], [51], [2248389.5]]
[['построят 153 ловочек, 8 тренажеров'], [273], [68773.421875]]


In [ ]:
# process('Сколько дворов участвуют в программе в ', 'B-GPE', 'I-GPE', 'город')

### NER

In [ ]:
# build - NER
ner_model = build_model(configs.ner.ner_ontonotes_bert_mult, download=True)

In [83]:
text = """Введут ли работы на ул. Загородная 165 и на Студенческой в г. Благовещенске?"""
print(ner_model([text])[0][0], ner_model([text])[1][0], sep='\n')

['Введут', 'ли', 'работы', 'на', 'ул', '.', 'Загородная', '165', 'и', 'на', 'Студенческой', 'в', 'г', '.', 'Благовещенске', '?']
['O', 'O', 'O', 'O', 'B-FAC', 'I-FAC', 'I-FAC', 'I-FAC', 'O', 'O', 'B-FAC', 'O', 'O', 'O', 'B-GPE', 'O']


In [93]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
text = """Введут ли работы на ул. Загородная 165 и на Студенческой в г. Благовещенске?"""

res = ner_model([text])
slice_ind = 0
slice_len = 1
entity = res[1][0]
token = res[0][0]

i = 0
while i < len(res[1][ 0]):
  if entity[i] == 'B-FAC':
    slice_ind = i
    slice_len = 1
    i += 1
    while entity[i] == 'I-FAC':
      slice_len += 1
      i += 1
    print(morph.parse(' '.join(token[slice_ind: slice_ind + slice_len]))[0].normal_form)
  i += 1

print(morph.parse('Загородная')[0].normal_form)
print(morph.parse('Загородной')[0].normal_form)
print(morph.parse('Загородную')[0].normal_form)

ул . загородная 165
студенческий
загородный
загородный
загородный


In [ ]:

def process(question, token_first='B-FAC', token_last='I-FAC', replace='улица N'):

  answer = ner_model([question])

  first = 0
  l = 0
  entity = list()

  for i, v in zip(range(1, len(answer[1][0]) + 1), answer[1][0]):

    if v == token_first:
      if l != 0:
        entity.append(answer[0][0][first - 1: first + l])
        question = question.replace(' '.join(answer[0][0][first - 1: first + l]), replace)
      first = i
      l = 0
    elif v == token_last:
      l += 1
    
    if i == len(answer[1][0]) and first:
      entity.append(answer[0][0][first - 1: first + l])
      question = question.replace(' '.join(answer[0][0][first - 1: first + l]), replace)
    

  return entity, question


process('Когда заканчиваются работы на улице Ленина 137?')

([['улице', 'Ленина', '137']], 'Когда заканчиваются работы на улица N?')

In [ ]:

a, b = process('Когда заканчиваются работы на улице Ленина 137?')
a[0][0] = 'ул'

In [ ]:
a

[['ул', 'Ленина', '137']]

### FAQ

In [ ]:
# build - FAQ 
model_config = read_json(configs.faq.tfidf_logreg_autofaq)
model_config['dataset_reader']['data_path'] = 'faq.csv'
model_config['dataset_reader']['data_url'] = None
faq = train_model(model_config);

2022-02-03 05:45:27.71 WARNING in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 219: Cannot load model from /root/.deeppavlov/models/vectorizer/tfidf_vectorizer_ruwiki_v2.pkl
2022-02-03 05:45:27.73 INFO in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 166: Initializing model sklearn.feature_extraction.text:TfidfVectorizer from scratch
2022-02-03 05:45:27.90 INFO in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 109: Fitting model sklearn.feature_extraction.text:TfidfVectorizer
2022-02-03 05:45:27.109 INFO in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 241: Saving model to /root/.deeppavlov/models/vectorizer/tfidf_vectorizer_ruwiki_v2.pkl
2022-02-03 05:45:27.119 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /root/.deeppavlov/models/faq/ru_mipt_answers.dict]
2022-02-03 05:45:27.122 WARNING in 'deeppavlov.models.sklearn.sklea

### Develov

In [ ]:
def greeting():
    print('/start')
    print('Привет, чем я могу помочь?')


def faq_func(question):
    state, score = faq(question)
    return state[0], not all(np.array(score[0]) < 0.5)


def odqa_document(question):
     return model_ru([text], [question])[0][0]


def ner_func(question):
    ent1, qu = process(question, token_first='B-FAC', token_last='I-FAC', replace='улица N')
    ent2, qu = process(qu, 'B-GPE', 'I-GPE', ' ')
    if ent1:
      ent1[0][0] = 'улица'
    if ent2:
      ent2[0].insert(0, 'город')
    return qu, ent1 + ent2


def sql_query_to_base():
    pass


In [ ]:
def main():

  greeting()

  dialog = True

  # Будем создавать словарь с параметрами для хранения контекста
  context = {}
  while dialog:
      question = input()

      if question == '/stop':
        return

      raw_question, entity = ner_func(question)
      if entity:
        for i in entity:
          context[i[0]] = ' '.join(i)
        
        
      print(raw_question)

      state, threshold = faq_func(raw_question)

      print(state, threshold)


      if threshold:

          if state == 'state1':
              # odqa_document(raw_question.replace('N', context['город']))
              if ''.join(context['улица'].split()[1:]) in ''.join(
                  process(text, token_first='B-FAC', token_last='I-FAC', replace='улица N')[0][0]):
                print('Участвует')
              else:
                print('Не участвует')
          elif state == 'state2':
              if not context and entity or context and entity:  # Если передали новую локацию - меняем контекст
                  context['loc'] = entity
              print(sql_query_to_base(context['loc']))  # Выполняем предопределнный запрос
          elif state == 'state3':
            print('')
          elif state =='state4':
            print('')
      else:
          # Иначе пытаемся найти вопрос в документах
          odqa_document(question)

In [ ]:
# main()

In [ ]:
ner_func('А участвует ли в программе улица Ленина 234 в городе Благовещенск?')

('А участвует ли в программе улица N в городе  ?',
 [['улица', 'Ленина', '234'], ['город', 'Благовещенск']])

In [ ]:
qu = ner_func('А участвует ли в программе улица Ленина 234 в городе Благовещенск?')
faq_func(qu)

TypeError: ignored

In [ ]:
faq(['А участвует ли в программе улица N в городе  ?'])

[['state1'],
 [[0.7478851319162771,
   0.0016191473149408913,
   0.24546254347542037,
   0.005033177293361457]]]

In [ ]:
model_ru([text], ['А участвует ли в программе улица Ленина 234 в городе Благовещенск?'])[0][0]

'более 100 дворов'

In [ ]:
context = {}
raw_question, entity = ner_func('А участвует ли в программе улица Ленина 234 в городе Благовещенск?')
if entity:
  for i in entity:
    context[i[0]] = ' '.join(i)

context

{'город': 'город Благовещенск', 'улица': 'улица Ленина 234'}

## Bot

In [ ]:
!pip install transformers
!pip install deeppavlov
!python -m deeppavlov install squad_bert

In [ ]:
import numpy as np

import deeppavlov
from deeppavlov import configs
from deeppavlov.core.common.file import read_json
from deeppavlov.core.commands.infer import build_model
from deeppavlov import configs, train_model

tkn_split = '>>>>>>>>>>>>'
text = ''' 
В 2022 года начата реализация программы “1000 дворов” в Амурской области. В течении года 
будет реализовано порядка 100 современных дворовых территорий. Все работы по созданию дворовых территорий будут 
завершены не позднее 30.11.2022 г. 
>>>>>>>>>>>>
Города которые участвуют в программе: Благовещенск, Белогорск, Тында, Циолковский, 
Шимановск, Райчихинск, Зея, Свободный.
>>>>>>>>>>>>
В Светлый, 6 будет установлено 10 лавочек, 5 качелей, 1 детский игровой комплекс, 3 карусели, 123 квадратных 
метра плитки, 44 метра леерного ограждения. На пер. Школьный, 110-112 будет установлено 8 лавочек, 6 качелей, 2 детских игровых комплексов, 
1 карусель, 144 квадратных метра плитки, 67 метра леерного ограждения.
>>>>>>>>>>>>
В Зее будет реализовано 5 дворовых территорий, это - мкр. Светлый, 6; мкр. Светлый, 61; мкр. Солнечный, 13,15; 
мкр. Светлый, 72, 73; пер. Школьный, 110-112.
В Благовещенске будет реализовано 55 дворовых территорий.
В Циолковском будет реализовано 2 дворовые территории.
В Тынде 7 дворовых территорий.
В Райчихинске будет реализовано 4 дворовые территории.
В Белогорске будет реализовано 6 дворовые территории.
В Свободном будет реализовано 21 дворовая территория.
В Шимановске будет реализовано 4 дворовые территории.
>>>>>>>>>>>>
ул.Театральная, д.35/1
ул.Трудовая, д.27,
ул.Амурская, д.114
ул.Загородная, д. 48, 
ул.Красноармейская, д.194
ул.Чайковского,д.211
ул.Чайковского, д.110
ул.Чайковского, 47
ул.Амурская, д.208
ул.Калинина,д.76
ул.Калинина, 76/2
ул.Ленина, д.12/2
ул.Ленина, д.14/2
ул.Строителей,д.79/3
ул.Моховая падь, ДОС-18
ул.50 лет Октября, д.203, ул.50 лет Октября, д,203/1, ул. Кольцевая,42А, ул. Островского, 236
ул.Ленина, д.171, ул.Б.Хмельницкого,д.6
ул.Моховая падь, Литер- 25
ул.Амурская,д.112
ул. Пионерская, 147, ул. 50 лет Октября 140, 142, 144
ул.Шевченко, 109
ул. Муравьева Амурского, 24, ул.Трудовая, 268
ул.Театральная,31, ул.Зейская,88
ул. Ленина,42
ул. Студенческая, 34/9
ул.Чайковского, 33
ул. Студенческая, 28
ул.Пионерская, 112/2
ул. Игнатьевское шоссе, 14, 14/1, 14/2, 14/4
ул.Горького,д. 97/1
ул. Игнатьевское шоссе, 3, 3/1
ул.Пушкина, д.3/ Краснофлотская д.53
ул. Ленина 235
ул. Политехническая 88
ул. Пограничная 124, 124/3
ул. Св.Иннокентия 19
ул. Дьяченко 2А,2В,2Г
ул. Игнатьевское шоссе 12/6
ул. Забурхановская 93, 93/1
ул. Шевченко 44
ул. Институтская 17, 17/1
ул. Мясокомбинат, Литер 1
ул. Красноармейская, 198
ул.Зейская,256
ул. 50 лет Октября, 197; ул. 50 лет Октября, 199;
ул. Институтская, 11
ул. Горького, 150
ул. Пушкина, 199/1
ул. Институтская 14
ул. Театральная, 94
ул. Чайковского, 64 
ул.Политехническая,48
ул.Горького,64
ул. Партизанская, 22/2
'''



model_ru = build_model(configs.squad.squad_ru_rubert, download=True)
ner_model = build_model(configs.ner.ner_ontonotes_bert_mult, download=True)
model_config = read_json(configs.faq.tfidf_logreg_autofaq)
model_config['dataset_reader']['data_path'] = 'faq.csv'
model_config['dataset_reader']['data_url'] = None
faq = train_model(model_config)


In [ ]:
text = '''В 2022 года начата реализация программы “1000 дворов” в Амурской области. В течении года 
будет реализовано порядка 100 современных дворовых территорий. Все работы по созданию дворовых территорий будут 
завершены не позднее 30.11.2022 г.
В Благовещенске на Ленина 123 будет установлено 10 лавочек, 5 качелей, 1 детский игровой комплекс, 3 карусели, 123 квадратных 
метра плитки, 44 метра леерного ограждения.
Города которые участвуют в программе: Благовещенск, Белогорск, Тында, Циолковский, 
Шимановск, Райчихинск, Зея, Свободный.'''

In [ ]:
# 

## 1

In [ ]:


def process(question, token_first='B-FAC', token_last='I-FAC', replace='улица N'):
    answer = ner_model([question])

    first = 0
    l = 0
    entity = list()

    for i, v in zip(range(1, len(answer[1][0]) + 1), answer[1][0]):

        if v == token_first:
            if l != 0 or token_first == 'B-GPE':
              entity.append(answer[0][0][first - 1: first + l])
              question = question.replace(' '.join(answer[0][0][first - 1: first + l]), replace)
            first = i
            l = 0
        elif v == token_last:
            l += 1

        if i == len(answer[1][0]) and first:
            entity.append(answer[0][0][first - 1: first + l])
            question = question.replace(' '.join(answer[0][0][first - 1: first + l]), replace)

    return entity, question


def greeting():
    print('/start')
    print('Привет, чем я могу помочь?')


# def faq_func(question):
#     state, score = faq(question)
#     return state[0], not all(np.array(score[0]) < 0.5)


def odqa_document(question):
    return model_ru([text], [question])[0][0]


def ner_func(question):
    ent1, qu = process(question, token_first='B-FAC', token_last='I-FAC', replace='улица N')
    ent2, qu = process(qu, 'B-GPE', 'I-GPE', ' ')
    d = {}
    if ent1:
        d['улица'] = ent1
    if ent2:
        d['город'] = ent2
    return qu, d


def sql_query_to_base():
    pass


def main():
    greeting()

    dialog = True

    # Будем создавать словарь с параметрами для хранения контекста
    context = {}
    while dialog:
        question = input()

        if question == '/stop':
            return

        raw_question, entity = ner_func(question)

        print(raw_question)

        # state, threshold = faq_func(raw_question)
        state, threshold = faq([question])

        state = state[0]
        threshold = not all(np.array(threshold[0]) < 0.5)

        print(state, threshold)

        if threshold:

            if state == 'state1':
                # odqa_document(raw_question.replace('N', context['город']))
                if ''.join(entity['улица'].split()[1:]) in ''.join(
                        process(text, token_first='B-FAC', token_last='I-FAC', replace='улица N')[0][0]):
                    print('Участвует')
                else:
                    print('Не участвует')
            elif state == 'state2':
                print(odqa_document(question))  # Выполняем предопределнный запрос
            elif state == 'state3':
                print(odqa_document(question))
            elif state == 'state4':
                print(odqa_document(question))
        else:
            # Иначе пытаемся найти вопрос в документах
            print(odqa_document(question))





## 2

In [ ]:
def process(question, token_first='B-FAC', token_last='I-FAC', replace='улица N'):
    answer = ner_model([question])

    first = 0
    l = 0
    entity = list()

    for i, v in zip(range(1, len(answer[1][0]) + 1), answer[1][0]):

        if v == token_first:
            if l != 0:
                entity.append(answer[0][0][first - 1: first + l])
                question = question.replace(' '.join(answer[0][0][first - 1: first + l]), replace)
            first = i
            l = 0
        elif v == token_last:
            l += 1

        if i == len(answer[1][0]) and first:
            entity.append(answer[0][0][first - 1: first + l])
            question = question.replace(' '.join(answer[0][0][first - 1: first + l]), replace)

    return entity, question


def greeting():
    print('/start')
    print('>>>>>>>>>>>>', 'Привет, чем я могу помочь?')


# def faq_func(question):
#     state, score = faq(question)
#     return state[0], not all(np.array(score[0]) < 0.5)


def odqa_document(question):
    return model_ru([text], [question])[0][0]


def ner_func(question):
    ent1, qu = process(question, token_first='B-FAC', token_last='I-FAC', replace='улица N')
    ent2, qu = process(qu, 'B-GPE', 'I-GPE', ' ')
    if ent1:
        ent1[0][0] = 'улица'
    if ent2:
        ent2[0].insert(0, 'город')
    return qu, ent1 + ent2


def sql_query_to_base():
    pass


def main():
    greeting()

    dialog = True

    # Будем создавать словарь с параметрами для хранения контекста
    context = {}
    while dialog:
        question = input()

        if question == '/stop':
            return

        raw_question, entity = ner_func(question)
        if entity:
            for i in entity:
                context[i[0]] = ' '.join(i)

        # print(raw_question)

        # state, threshold = faq_func(raw_question)
        state, threshold = faq([question])

        state = state[0]
        threshold = not all(np.array(threshold[0]) < 0.5)

        # print(state, threshold)

        if threshold:

            if state == 'state1':
                # odqa_document(raw_question.replace('N', context['город']))
                if ''.join(context['улица'].split()[1:]) in ''.join(
                        process(text, token_first='B-FAC', token_last='I-FAC', replace='улица N')[0][0]):
                    print('>>>>>>>>>>>>', 'Улица', 'участвует')
                else:
                    print('>>>>>>>>>>>>', 'Улица', 'не участвует')
            elif state == 'state2':
                print('>>>>>>>>>>>>', odqa_document(question).title())  # Выполняем предопределнный запрос
            elif state == 'state3':
                print('>>>>>>>>>>>>', odqa_document(question).title())
            elif state == 'state4':
                print(odqa_document(question))
        else:
            # Иначе пытаемся найти вопрос в документах
            print('>>>>>>>>>>>>', odqa_document(question).title())



## main

In [ ]:
text = ''' В этом году в Благовещенске участвуют в программе более 100 дворов. 12.24.2021 завершаются ремонтные 
работы на улице Ленина 165. Академический театр драмы обновляется в соответствии с решением губернатора. Также 
13.34.2035 начинаются работы на улице Богдана-Хмельницкого 156, построят 153 ловочек, 8 тренажеров.'''


In [ ]:
main()

/start
>>>>>>>>>>>> Привет, чем я могу помочь?
Привет, Участвует двор на улице Ленина 1456 в программе?
>>>>>>>>>>>> Улица не участвует
Участвует ли двор на Ленина 165 в программе?
>>>>>>>>>>>> Улица участвует
Сколько дворов участвет в программе в этом году?
>>>>>>>>>>>> Более 100 Дворов
Что будет сделано на Богдана-Хмельницкого 156?
>>>>>>>>>>>> Построят 153 Ловочек, 8 Тренажеров
Когда начинаются работы на улице Богдана-Хмельницкого?
>>>>>>>>>>>> 13.34.2035
/stop


In [ ]:
def process(question, token_first='B-FAC', token_last='I-FAC', replace='улица N'):
    answer = ner_model([question])

    first = 0
    l = 0
    entity = list()

    for i, v in zip(range(1, len(answer[1][0]) + 1), answer[1][0]):

        if v == token_first:
            if l != 0:
                entity.append(answer[0][0][first - 1: first + l])
                question = question.replace(' '.join(answer[0][0][first - 1: first + l]), replace)
            first = i
            l = 0
        elif v == token_last:
            l += 1

        if i == len(answer[1][0]) and first:
            entity.append(answer[0][0][first - 1: first + l])
            question = question.replace(' '.join(answer[0][0][first - 1: first + l]), replace)

    return entity, question
process('улице Ленина 145')

([], 'улице Ленина 145')

In [ ]:
ner_model(['Привет, Участвует двор на улице Ленина 1456 в программе?'])

(['Привет, Участвует двор на улице Ленина 1456 в программе?'], {})

In [ ]:
txt_dict = dict()
for txt, v in zip(text.split(tkn_split),
                  ['date', 'city', 'project', 'projects', 'streets']):
    txt_dict[v] = txt

[['10 лавочек, 5 качелей, 1 детский игровой комплекс, 3 карусели'],
 [28],
 [260332.046875]]

{'city': '\nГорода которые участвуют в программе: Благовещенск, Белогорск, Тында, Циолковский, \nШимановск, Райчихинск, Зея, Свободный.\n',
 'date': ' \nВ 2022 года начата реализация программы “1000 дворов” в Амурской области. В течении года \nбудет реализовано порядка 100 современных дворовых территорий. Все работы по созданию дворовых территорий будут \nзавершены не позднее 30.11.2022 г. \n',
 'project': '\nВ Зее будет реализовано 5 дворовых территорий, это - мкр. Светлый, 6; мкр. Светлый, 61; мкр. Солнечный, 13,15; \nмкр. Светлый, 72, 73; пер. Школьный, 110-112.\n',
 'projects': '\nВ Светлый, 6 будет установлено 10 лавочек, 5 качелей, 1 детский игровой комплекс, 3 карусели, 123 квадратных \nметра плитки, 44 метра леерного ограждения. На пер. Школьный, 110-112 будет установлено 8 лавочек, 6 качелей, 2 детских игровых комплексов, \n1 карусель, 144 квадратных метра плитки, 67 метра леерного ограждения.\n',
 'streets': '\nВ Благовещенске будет реализовано 55 дворовых территорий.\nВ Циол

# KMU


In [ ]:
!pip install transformers
!pip install deeppavlov
!python -m deeppavlov install squad_bert

In [ ]:
!pip install pymorphy2
!pip install pymorphy2-dicts
!pip install DAWG-Python

In [121]:
from deeppavlov.metrics.accuracy import sets_accuracy
sets_accuracy(pd.read_csv('faq.csv')['Answer'], faq(pd.read_csv('faq.csv')['Question'])[0])

1.0

In [ ]:
squad = build_model(configs.squad.squad_ru_rubert, download=True)

In [ ]:
# build - FAQ
import pandas as pd
# !wget https://raw.githubusercontent.com/AraKchrUser/Automation-of-citizens-appeals/master/faq.csv
model_config = read_json(configs.faq.tfidf_logreg_autofaq)
model_config['dataset_reader']['data_path'] = 'faq.csv'
model_config['dataset_reader']['data_url'] = None
# model_config['train'] = OrderedDict([('class_name', 'fit_trainer'), ('metrics', ['accuracy']), ('train_score', ['accuracy'])])
faq = train_model(model_config);

In [93]:
# build - NER
ner_model = build_model(configs.ner.ner_ontonotes_bert_mult, download=True)

In [241]:
def preprocess(text, tokens=('B-FAC', 'I-FAC')):
  res = ner_model([text])
  tkn_1, tkn_2 = tokens
  value = list()

  slice_ind = 0
  slice_len = 1
  entity = res[1][0]
  token = res[0][0]

  i = 0
  while i < len(res[1][ 0]):
    if entity[i] == tkn_1:
      slice_ind = i
      slice_len = 1
      i += 1
      while i != len(res[1][ 0]) and entity[i] == tkn_2:
        slice_len += 1
        i += 1
      value.append(normalization(token[slice_ind: slice_ind + slice_len]))
    i += 1
  return value

text = """Введут ли работы на ул. Фрунзе 123 и на пер. Студенческой 12 в г. Благовещенске?"""
preprocess(text, ('B-FAC', 'I-FAC'))

['ул . фрунзе 123', 'пер . студенческий 12']

In [160]:
def normalization(tokens):
  norm_tokens = []
  if isinstance(tokens, str):
    tokens = tokens.split()
  for token in tokens:
    norm_tokens.append(morph.parse(token)[0].normal_form)
  return ' '.join(norm_tokens)

In [173]:
# print(morph.parse('Фетисова')[0].normal_form)
# print(morph.parse('Фетисовой')[0].normal_form)
# print(morph.parse('Загородную')[0].normal_form)

In [ ]:
!pip uninstall pandas
!pip install pandas

In [133]:
# !wget https://raw.githubusercontent.com/AraKchrUser/Automation-of-citizens-appeals/master/db.csv
data = pd.read_csv('db.csv', sep=';')

In [309]:
txt1 = """110-112 будет установлено 8 лавочек, 6 качелей, 2 детских игровых комплексов, 
1 карусель, 144 квадратных метра плитки, 67 метра леерного ограждения."""
txt2 = """ будет установлено 10 лавочек, 5 качелей, 1 детский игровой комплекс, 
3 карусели, 123 квадратных метра плитки, 44 метра леерного ограждения."""

In [314]:
data['proj'] = None
data.iloc[0, -1] = txt1
data.iloc[2, -1] = txt2
data

,город,адрес,статус,сроки,norm_address,norm_city,proj
0,Тында,Студенческая 21,в реализации,12.12.22 - 01.02.23,студенческий 21,тында,"110-112 будет установлено 8 лавочек, 6 качелей..."
1,Благовещенск,Пушкина-Колотушкина 165,в реализации,10.10.22 - 01.02.23,пушкин - колотушкин 165,благовещенск,None
2,Сковородино,Светителя 123,в реализации,09.09.22 - 01.02.23,светителя 123,сковородино,"будет установлено 10 лавочек, 5 качелей, 1 де..."
3,Благовещенск,Ленина 12,закрыта,08.08.22 - 01.02.23,ленин 12,благовещенск,None
4,Тында,Армена 007,в реализации,07.07.22 - 01.02.23,армен 007,тында,None
5,Шимановск,Русская 69,закрыта,06.06.22 - 01.02.23,русский 69,шимановск,None
6,Вожаевка,Японская 666,закрыта,05.05.22 - 01.02.23,японский 666,вожаевка,None
7,Благовещенск,Фрунзе 109,закрыта,04.04.22 - 01.02.23,фрунзе 109,благовещенск,None
8,Благовещенск,Фетисова 120,в реализации,03.03.22 - 01.02.23,фетисов 120,благовещенск,None


In [354]:
def greeting():
  print('/start', 'Привет \U0001F44B, чем я могу помочь?', sep='\n')


def fill_street(question, context):
  street = preprocess(question, tokens=('B-FAC', 'I-FAC'))
  if street:
      context['street'] = address_prepr(street[0])


def fill_city(question, context): 
  city = preprocess(question, tokens=('B-GPE', 'I-GPE'))
  if city:
    context['city'] = city[0]


def check_city(context):
  while not context['city']:
    print('Вы не ввели город \U0001F609')
    question = input()
    fill_city(question, context)

def check_street(context):
  while not context['street']:
    print('Вы не ввели улицу \U0001F64F')
    question = input()
    fill_street(question, context)
    


def address_prepr(text):
  text = text.replace('улица', '').strip()
  text = text.replace('ул.', '').strip()
  return text



context = {'city': None, 'street': None}
data['norm_address'] = data['адрес'].apply(normalization).apply(lambda x: x.replace('-', ' - ').strip())
data['norm_city'] = data['город'].apply(normalization)
text = ''
quary = ''


def main():
  dialog = True
  greeting()

  while dialog:

    question = input()
    if question == '/stop':
      return
    if question == '+':
      ## Тут будут конкретные вопросы-ответы о наполняемости  
      while question != '-':
        question = input()
        fill_street(question, context)
        fill_city(question, context)
        check_city(context)
        check_street(context)
        quary = data[data['norm_city'] == context['city']]
        text = quary[quary['norm_address'] == context['street']]['proj'].values.tolist()
        if text:
          print(squad(text, [question])[0][0])
        else:
          print('Двор не участвует в программе')
        
    
    fill_street(question, context)
    fill_city(question, context)
  
    state, threshold = faq([question])

    state = state[0]
    threshold = not all(np.array(threshold[0]) < 0.93)

    if not threshold:
      print('Кто такой Илон Маск, говоришь?')
    elif state == 'state1':
      check_city(context)
      check_street(context)
      if context['street'] in data[data['norm_city'] == context['city']]['norm_address'].tolist():
        print('Двор участвует в программе')
      else:
        print('Двор не участвует в программе')
    elif state == 'state2':
      check_city(context)
      check_street(context)
      strt = data[data['norm_city'] == context['city']]
      strt = strt[strt['norm_address'] == context['street']]['сроки'].values
      if strt:
        print(context['street'], ": ", strt[0])
      else:
        print("Двор не участвует в программе")
    elif state == 'state3':
      print(len(pd.unique(data['norm_address'])))

In [355]:
main()

/start
Привет 👋, чем я могу помочь?
+
Сколько качелей будет установлено в Тынде на улице Студенческая 22?
Двор не участвует в программе
-
Двор не участвует в программе
Кто такой Илон Маск, говоришь?
Сколько качелей будет установлено в Тынде на улице Студенческая 22?
Кто такой Илон Маск, говоришь?
Участвует двор на улице Японская 666 в Вожаевке в программе?
Кто такой Илон Маск, говоришь?
Участвует двор на улице Японская 666 в Благовещенске в программе?
Кто такой Илон Маск, говоришь?
/stop


In [339]:
quary = data[data['norm_city'] == context['city']]
quary

,город,адрес,статус,сроки,norm_address,norm_city,proj
0,Тында,Студенческая 21,в реализации,12.12.22 - 01.02.23,студенческий 21,тында,"110-112 будет установлено 8 лавочек, 6 качелей..."
4,Тында,Армена 007,в реализации,07.07.22 - 01.02.23,армен 007,тында,None


In [341]:
text = quary[quary['norm_address'] == context['street']]['proj']
text.value

0    110-112 будет установлено 8 лавочек, 6 качелей...
Name: proj, dtype: object

In [356]:
faq(['Участвует двор на улице Японская 666 в Вожаевке в программе?'])

[['state1'],
 [[0.7923871256607374,
   0.006429901927371733,
   0.1980862487628603,
   0.003096723649030567]]]

In [ ]:
## !!< Баг!

In [351]:
faq(['Сколько качелей будет установлено в Тынде на улице Студенческая 21?'])

[['state3'],
 [[0.0009557733961661296,
   0.025371147100208224,
   0.9298621232677957,
   0.04381095623582994]]]

In [288]:
main()

/start
Привет 👋, чем я могу помочь?
Привет, подскажи, участвует ли двор на улице Ленина 145 в программе?
Вы не ввели город 😉
Давай Благовещенск
Вы не ввели город 😉
Благовещенск
Двор не участвует в программе
участвует ли двор на улице Ленина 12 в программе?
Двор участвует в программе
Участвует ли в программе улица Студенческая 21?
Двор не участвует в программе
Участвует ли в программе улица Студенческая 21  в Тынде?
Двор участвует в программе
Сколько дворов участвует в программе?
9
Когда закончится обустройство на Студенческой 21?
студенческий 21 :  12.12.22 - 01.02.23
Когда закончатся работы на улице Фрунзе 109 в Благовещенске?
фрунзе 109 :  04.04.22 - 01.02.23
/stop


In [302]:
text = """На Ленина 110-112 будет установлено 8 лавочек, 6 качелей, 2 детских игровых комплексов, 
1 карусель, 144 квадратных метра плитки, 67 метра леерного ограждения."""
squad([text], ['Сколько будет установлено качелей во дворе на улице Ленина 196?'])[0][0]


'6'

In [293]:
text[36:]

'8 лавочек, 6 качелей, 2 детских игровых комплексов, \n1 карусель, 144 квадратных метра плитки, 67 метра леерного ограждения.'

In [303]:
state, threshold = faq([question])

In [305]:
threshold

[[0.004134446011396008,
  0.9767034124644673,
  0.005428488541833146,
  0.013733652982303614]]